In [419]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [420]:
import pandas as pd
import numpy as np
from ambiente import usedAmbiente

In [421]:
coordPoints = pd.read_csv(usedAmbiente.coord, sep='\t', header=None, names=['id', 'coord'])

In [422]:
trace = pd.read_csv(usedAmbiente.trace, sep='\t', header=None, names=['timestamp', 'transmissor', 'receptor', 'snr'])


In [423]:
trace = trace.sort_values(['receptor','transmissor','timestamp']) #ordena na ordem dos parãmetros
aps = ['00:00:00:00:00:01','00:00:00:00:00:02','00:00:00:00:00:03','00:00:00:00:00:04','00:00:00:00:00:05','00:00:00:00:00:06','00:00:00:00:00:07']
trace = trace[~trace.receptor.isin(aps)]

In [424]:
coordPoints = coordPoints.iloc[7:]

In [425]:
trace['time'] = np.floor(trace.timestamp) #corta o tempo em inteiro


In [426]:
groupedByTime = trace.groupby(['receptor','transmissor','time'], as_index= False).agg({'snr':'mean'}) #calcula a media do snr para cada par receptor-transmissor no tempo X

In [427]:
groupedByReceptor = groupedByTime.pivot(columns='transmissor', index=['receptor','time'],values='snr') #indexa o snr de cada transmissor ao par receptor-tempo

In [428]:
groupedByReceptor= groupedByReceptor-98 # calcula o dbm

In [429]:
#groupedByReceptor.to_csv('groupedByReceptor.tsv', '\t')

In [430]:
groupedByReceptor = groupedByReceptor.dropna() #apaga todos os NAN
#groupedByReceptor = groupedByReceptor.fillna(0) #substitui todos os NAN
groupedByReceptor = groupedByReceptor.apply(lambda line: ','.join(line.astype(str)),axis=1) #junta todos os dbm para formar o vetor fingerprint
groupedByReceptor = groupedByReceptor.rename('fingerprint') 

In [431]:
coordPoints.id +=8 #acerta o index para que a contagem de transmissores e receptores seja continua
coordPoints.id = coordPoints.id.map(lambda id : '00:00:00:00:00:{:02X}'.format(id).lower()) #formata e converte o index para HEX

In [432]:
groupedByReceptor= groupedByReceptor.reset_index() #receptor e time voltam a ser colunas

In [433]:
fingerprint1 = groupedByReceptor.merge(coordPoints,how='left',  left_on='receptor', right_on='id') #juntas as duas tabelas

In [434]:
#fingerprint1[fingerprint1['id'].isna()].receptor.unique()
fingerprint1 = fingerprint1[['coord', 'fingerprint']] #remove os campos extras

In [435]:
fingerprint1.to_csv(usedAmbiente.exitFile, '\t', index=False, header=False)

C:\Users\mayla\AppData\Local\Temp\ipykernel_5820\3662866187.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_csv except for the argument 'path_or_buf' will be keyword-only.
  fingerprint1.to_csv(usedAmbiente.exitFile, '\t', index=False, header=False)
